# PyTorch Experiments to predict Grit Mindset

## Import Libraries

In [2]:
# install skorch, a scikilt learn wrapper for PyTorch
! pip install -U skorch

     |████████████████████████████████| 102kB 592kB/s ta 0:00:01


In [1]:
import torch
import torch.nn as nn
from torch.autograd import Variable
import torch.optim as optim

import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, StratifiedShuffleSplit
import os
import sys
import glob
from sklearn.utils import shuffle
import matplotlib.pyplot as plt
from sklearn import model_selection
from sklearn.metrics import accuracy_score, precision_score, recall_score
import matplotlib.pyplot as plt
from skorch import NeuralNetClassifier
from sklearn.model_selection import GridSearchCV

### Note Notebook doesn't display all the rows and columns
- let's fix that

In [2]:
pd.options.display.max_columns = None
pd.options.display.max_rows = None

## Check CSV file inside data folder

In [3]:
! ls data

GritMindset.csv                      models
GritMindset.csvDroppedNaNCols.pickle


In [4]:
def loadData(csvFile):
    pickleDump = '{}DroppedNaNCols.pickle'.format(csvFile)
    if os.path.exists(pickleDump):
        df = pd.read_pickle(pickleDump)
    else:
        df = pd.read_csv(csvFile, low_memory=False)
        # clean data
        # strip the whitspaces from column names
        df = df.rename(str.strip, axis='columns')
        # drop Infinity rows and NaN string from each column
        for col in df.columns:
            indexNames = df[df[col] == 'Infinity'].index
            if not indexNames.empty:
                print('deleting {} rows with Infinity in column {}'.format(len(indexNames), col))
                df.drop(indexNames, inplace=True)
            
        # drop all columns with NaN values
        beforeColumns = df.shape[1]
        df.dropna(axis='columns', inplace=True)
        print('Dropped {} columns with NaN values'.format(beforeColumns - df.shape[1]))
        # drop all rows with NaN values
        beforeRows = df.shape[0]
        df.dropna(inplace=True)
        print('Dropped {} rows with NaN values'.format(beforeRows - df.shape[0]))
        df.to_pickle(pickleDump)
        
    return df

In [5]:
df = loadData('data/GritMindset.csv')

In [6]:
# let's check the shape again
df.shape

(309, 4)

In [7]:
# display first 5 samples
df.head()

,Gender,Age,HighSchoolGrade,HonorsScience
0,1,19,85.0,2
1,2,19,94.0,1
2,1,19,85.0,2
3,2,19,80.0,2
4,2,18,78.0,2


In [8]:
df.columns

Index(['Gender', 'Age', 'HighSchoolGrade', 'HonorsScience'], dtype='object')

In [9]:
# class distribution
label = 'HonorsScience'
print(df.groupby(label).size())

HonorsScience
1     48
2    261
dtype: int64


## Binary-class classification
- Relabel HonorsScience 1 as 0 and 2 as 1

In [10]:
lblTypes = set(df[label])
lblTypes

{1, 2}

In [11]:
lblTypes = dict(zip(lblTypes, [0]*2))
lblTypes[2] = 1
lblTypes

{1: 0, 2: 1}

In [12]:
df[label] = df[label].map(lblTypes)

In [13]:
df.head()

,Gender,Age,HighSchoolGrade,HonorsScience
0,1,19,85.0,1
1,2,19,94.0,0
2,1,19,85.0,1
3,2,19,80.0,1
4,2,18,78.0,1


In [14]:
sss = StratifiedShuffleSplit(n_splits = 10, test_size=0.2, random_state=0)
print(sss)

StratifiedShuffleSplit(n_splits=10, random_state=0, test_size=0.2,
            train_size=None)


In [15]:
class Net(nn.Module):
    def __init__(self):
        # define nn
        super().__init__()
        self.inputLayer = nn.Linear(3, 50)
        self.relu1 = nn.ReLU()
        #self.dout = nn.Dropout(0.2)
        self.layer2 = nn.Linear(50, 100)
        self.prelu = nn.PReLU(1)
        self.out = nn.Linear(100, 2)
        self.out_act = nn.Sigmoid()
    
    def forward(self, X):
        X = self.inputLayer(X)
        X = self.relu1(X)
        #X = self.dout(X)
        X = self.layer2(X)
        X = self.prelu(X)
        X = self.out(X)
        y = self.out_act(X)
        return y
    

In [16]:
data_y = df.pop("HonorsScience").values
data_X = df.values

In [17]:
data_X = data_X.astype(np.float32)
data_y = data_y.astype(np.int64)
params = {
    'lr': [0.01, 0.02],
    'max_epochs': [10, 100],
    'module_num_units': [10,20],
}

In [20]:
def gridSearch(model):
    gs = GridSearchCV(model, params, refit=False, cv=10, scoring='accuracy')
    gs.fit(data_X, data_y)
    return gs.best_score_, gs.best_params_

In [21]:
"""
model = Net()
opt = optim.Adam(net.parameters(), lr=0.001, betas=(0.9, 0.999))
criterion = nn.BCELoss()
"""
opts = [optim.Adadelta, optim.Adagrad, optim.Adam, optim.RMSprop, optim.SGD]
for opt in opts:
    model = NeuralNetClassifier(Net, optimizer=opt, max_epochs=100, lr=0.1, iterator_train__shuffle=True,)
    print(str(opt), *gridSearch(model))

  epoch    train_loss    valid_acc    valid_loss     dur
-------  ------------  -----------  ------------  ------
      1        2.3797       0.2143        2.2141  0.0141
      2        2.2482       0.2143        2.0842  0.0145
      3        2.1161       0.2143        1.9556  0.0196
      4        1.9860       0.2143        1.8301  0.0168
      5        1.8583       0.2143        1.7075  0.0184
      6        1.7344       0.2143        1.5895  0.0156
      7        1.6147       0.2321        1.4760  0.0489
      8        1.4999       0.2500        1.3679  0.0476
      9        1.3907       0.2679        1.2658  0.0143
     10        1.2875       0.2679        1.1697  0.0194
  epoch    train_loss    valid_acc    valid_loss     dur
-------  ------------  -----------  ------------  ------
      1        2.4848       0.8393        2.4247  0.0155
      2        2.3669       0.8393        2.3152  0.0548
      3        2.2534       0.8393        2.2119  0.0218
      4        2.1461       0.8

      4        9.3415       0.1607        8.6990  0.0591
      5        9.1718       0.1607        8.5385  0.0256
      6        9.0003       0.1607        8.3758  0.0134
      7        8.8272       0.1607        8.2122  0.0190
      8        8.6522       0.1607        8.0463  0.0131
      9        8.4758       0.1607        7.8794  0.0142
     10        8.2982       0.1607        7.7116  0.0190
  epoch    train_loss    valid_acc    valid_loss     dur
-------  ------------  -----------  ------------  ------
      1        4.0892       0.1607        3.8531  0.0132
      2        3.9424       0.1607        3.7051  0.0149
      3        3.7923       0.1607        3.5555  0.0141
      4        3.6390       0.1607        3.4028  0.0135
      5        3.4835       0.1607        3.2490  0.0141
      6        3.3267       0.1607        3.0944  0.0147
      7        3.1693       0.1607        2.9394  0.0141
      8        3.0115       0.1607        2.7845  0.0139
      9        2.8531       0.1

      9        3.1189       0.1607        2.8374  0.0136
     10        2.9862       0.1607        2.7124  0.0141
  epoch    train_loss    valid_acc    valid_loss     dur
-------  ------------  -----------  ------------  ------
      1        0.7956       0.8393        0.7783  0.0139
      2        0.7611       0.8393        0.7429  0.0148
      3        0.7280       0.8393        0.7097  0.0139
      4        0.6944       0.8393        0.6750  0.0140
      5        0.6597       0.8393        0.6392  0.0163
      6        0.6240       0.8393        0.6023  0.0158
      7        0.5884       0.8393        0.5669  0.0138
      8        0.5522       0.8393        0.5300  0.0140
      9        0.5164       0.8393        0.4948  0.0160
     10        0.4815       0.8393        0.4601  0.0502
  epoch    train_loss    valid_acc    valid_loss     dur
-------  ------------  -----------  ------------  ------
      1        1.0477       0.1607        0.9303  0.0150
      2        0.9478       0.1

     93        0.0118       0.7857        0.0039  0.0140
     94        0.0115       0.7857        0.0037  0.0134
     95        0.0112       0.7857        0.0035  0.0139
     96        0.0109       0.7857        0.0034  0.0140
     97        0.0106       0.7857        0.0032  0.0129
     98        0.0104       0.7857        0.0031  0.0140
     99        0.0101       0.7857        0.0029  0.0128
    100        0.0098       0.8036        0.0028  0.0129
  epoch    train_loss    valid_acc    valid_loss     dur
-------  ------------  -----------  ------------  ------
      1        0.1164       0.8393        0.1337  0.0572
      2        0.1044       0.8393        0.1217  0.0143
      3        0.0938       0.8393        0.1111  0.0139
      4        0.0847       0.8393        0.1021  0.0185
      5        0.0770       0.8393        0.0945  0.0147
      6        0.0705       0.8393        0.0877  0.0218
      7        0.0648       0.8393        0.0820  0.0157
      8        0.0600       0.8

  epoch    train_loss    valid_acc    valid_loss     dur
-------  ------------  -----------  ------------  ------
      1        0.4157       0.2143        0.3449  0.0600
      2        0.3461       0.2143        0.2885  0.0351
      3        0.2894       0.2143        0.2437  0.0173
      4        0.2450       0.2143        0.2089  0.0173
      5        0.2101       0.2143        0.1810  0.0179
      6        0.1823       0.2143        0.1588  0.0322
      7        0.1598       0.2143        0.1405  0.0162
      8        0.1414       0.2143        0.1254  0.0228
      9        0.1261       0.2143        0.1127  0.0144
     10        0.1133       0.2143        0.1021  0.0153
     11        0.1025       0.2143        0.0929  0.0136
     12        0.0932       0.2143        0.0850  0.0159
     13        0.0852       0.2143        0.0781  0.0140
     14        0.0782       0.2143        0.0721  0.0151
     15        0.0721       0.2143        0.0667  0.0140
     16        0.0666       0.2

...

     55        0.0001       0.8393        0.0000  0.0121
     56        0.0001       0.8393        0.0000  0.0113
     57        0.0001       0.8393        0.0000  0.0112
     58        0.0001       0.8393        0.0000  0.0115
     59        0.0001       0.8393        0.0000  0.0593
     60        0.0001       0.8393        0.0000  0.0122
     61        0.0001       0.8393        0.0000  0.0119
     62        0.0001       0.8393        0.0000  0.0112
     63        0.0001       0.8393        0.0000  0.0127
     64        0.0001       0.8393        0.0000  0.0364
     65        0.0001       0.8393        0.0000  0.0193
     66        0.0001       0.8393        0.0000  0.0114
     67        0.0001       0.8393        0.0000  0.0120
     68        0.0001       0.8393        0.0000  0.0114
     69        0.0001       0.8393        0.0000  0.0115
     70        0.0001       0.8393        0.0000  0.0114
     71        0.0001       0.8393        0.0000  0.0117
     72        0.0001       0.8

     68        0.0003       0.8393        0.0016  0.0133
     69        0.0003       0.8393        0.0016  0.0119
     70        0.0003       0.8393        0.0016  0.0114
     71        0.0003       0.8393        0.0016  0.0134
     72        0.0003       0.8393        0.0016  0.0123
     73        0.0003       0.8393        0.0016  0.0129
     74        0.0003       0.8393        0.0016  0.0178
     75        0.0003       0.8393        0.0015  0.0127
     76        0.0003       0.8393        0.0015  0.0117
     77        0.0003       0.8393        0.0015  0.0112
     78        0.0003       0.8393        0.0015  0.0114
     79        0.0003       0.8393        0.0015  0.0112
     80        0.0003       0.8393        0.0015  0.0119
     81        0.0003       0.8393        0.0015  0.0119
     82        0.0003       0.8393        0.0015  0.0141
     83        0.0003       0.8393        0.0014  0.0159
     84        0.0003       0.8393        0.0014  0.0133
     85        0.0003       0.8

     76        0.0004       0.1607        0.0041  0.0108
     77        0.0004       0.1607        0.0041  0.0113
     78        0.0004       0.1607        0.0040  0.0117
     79        0.0004       0.1607        0.0040  0.0125
     80        0.0004       0.1607        0.0040  0.0127
     81        0.0004       0.1607        0.0039  0.0114
     82        0.0004       0.1607        0.0039  0.0113
     83        0.0004       0.1607        0.0039  0.0117
     84        0.0004       0.1607        0.0038  0.0117
     85        0.0004       0.1607        0.0038  0.0109
     86        0.0004       0.1607        0.0038  0.0101
     87        0.0004       0.1607        0.0037  0.0106
     88        0.0003       0.1607        0.0037  0.0103
     89        0.0003       0.1607        0.0037  0.0113
     90        0.0003       0.1607        0.0036  0.0112
     91        0.0003       0.1607        0.0036  0.0109
     92        0.0003       0.1607        0.0036  0.0110
     93        0.0003       0.1

     84        0.0000       0.8393        0.0002  0.0112
     85        0.0000       0.8393        0.0002  0.0220
     86        0.0000       0.8393        0.0002  0.0118
     87        0.0000       0.8393        0.0002  0.0116
     88        0.0000       0.8393        0.0002  0.0116
     89        0.0000       0.8393        0.0002  0.0120
     90        0.0000       0.8393        0.0002  0.0110
     91        0.0000       0.8393        0.0002  0.0113
     92        0.0000       0.8393        0.0002  0.0108
     93        0.0000       0.8393        0.0002  0.0111
     94        0.0000       0.8393        0.0002  0.0112
     95        0.0000       0.8393        0.0002  0.0114
     96        0.0000       0.8393        0.0002  0.0109
     97        0.0000       0.8393        0.0002  0.0109
     98        0.0000       0.8393        0.0002  0.0108
     99        0.0000       0.8393        0.0002  0.0113
    100        0.0000       0.8393        0.0002  0.0100
<class 'torch.optim.sgd.SGD'> 0

/Users/rbasnet/miniconda3/lib/python3.7/site-packages/sklearn/model_selection/_search.py:813: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)
